In [ ]:
import os
os.chdir('/home/fsboedij/Data/MouseAtlas')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd
import pandas as pd
from pySankey.sankey import sankey
from matplotlib.pyplot import rc_context
import seaborn as sns
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(5, 5))
torch.set_printoptions(precision=10, sci_mode=False, edgeitems=12)
pd.options.mode.chained_assignment = None
import matplotlib.colors as mcolors
import utils
from sklearn.linear_model import LinearRegression
import matplotlib.cm as cm


In [ ]:
adata=sc.read('Data/Integration/Step10_fulladata_final_0411.h5ad')
print(adata)

In [ ]:

adata = adata[adata.obs['healthy_or_disease'] == 'Healthy']

#Taking out the 'Post-Sendai' set due to the large epi composition

adata = adata[adata.obs['study'] != 'Mouse_Post_Sendai']
#full_latent = sc.read("/data/kkovacs/Python/Data/Kyle_Atlas_emb.h5ad")

#full_latent.obs_names_make_unique()

#full_latent = full_latent[adata.obs_names,:]
#adata.obsm['X_MLCA'] = full_latent.X.copy()

adata.X.data=adata.layers['raw'].data.copy()

dir_output='/home/fsboedij/Data/MouseAtlas/Data/others/'
dir_results=dir_output

FIGURES = dict()

viral_to_cont = {"Viral": 1, "Post-Viral": 0.5, "Non-Viral": 0}
adata.obs["viral_status_num"] = adata.obs.Viral.map(viral_to_cont).astype(float)

sex_to_cont = {"Male": 1, "Female": 0}
adata.obs["sex_cont"] = adata.obs.sex.map(sex_to_cont).astype('category')

adata.obs['age_cont'] = adata.obs['age']
adata.obs['age'] = adata.obs['age'].astype(object)
#adata.obs['age_cont'].loc[adata.obs['age_cont'].isin(['1','5-6','6-8','8','10-12'])] = 'Young'
#adata.obs['age_cont'].loc[adata.obs['age_cont'].isin(['12','6-20','12-13','16-20','13','14'])] = 'Adult'
#adata.obs['age_cont'].loc[adata.obs['age_cont'].isin(['18-20','78','104'])] = 'Old'
adata.obs['age'].loc[adata.obs['age'].isin(['5-6'])] = '5.5'
adata.obs['age'].loc[adata.obs['age'].isin(['6-8'])] = '7'
adata.obs['age'].loc[adata.obs['age'].isin(['10-12'])] = '11'
adata.obs['age'].loc[adata.obs['age'].isin(['6-20'])] = '13'
adata.obs['age'].loc[adata.obs['age'].isin(['12-13'])] = '12.5'
adata.obs['age'].loc[adata.obs['age'].isin(['16-20'])] = '18'
adata.obs['age'].loc[adata.obs['age'].isin(['18-20'])] = '19'
adata.obs['age'] = adata.obs['age'].astype(float)
print(adata.obs['age'].value_counts())


adata.obs['pct_counts_mt']=adata.obs['percent.mt'].copy()
adata.obs['pct_counts_mt'] = adata.obs['pct_counts_mt'].astype(float)

adata.obs['total_counts'] = adata.obs['total_counts'].astype(float)

adata.obs['Level_3_5']=adata.obs['Level_3'].copy()
adata.obs['Level_3_5']=adata.obs['Level_3_5'].astype(object)
adata.obs['Level_3_5'].loc[(adata.obs['Level_3'].isin([ 'T cell lineage']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Mast cells']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Club_Goblet']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Smoke induced club_goblet']))] = 'Club_Goblet'
adata.obs['Level_3_5'].loc[(adata.obs['Level_5'].isin([ 'Lymphatic endothelial']))] = adata.obs.Level_5
#adata.obs['Level_3_5'].loc[(adata.obs['Level_3'].isin([ 'Dendritic cells']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_5'].isin([ 'pDC']))] = adata.obs.Level_5
#adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Neutrophils']))] = adata.obs.Level_5
#adata.obs['Level_3_5'].loc[(adata.obs['Level_3_5'].isin([ 'Proliferating neutrophils']))] ='Neutrophils'

#adata.obs['Level_3_5'].loc[(adata.obs['Level_3'].isin([ 'Capillary endothelial','Venous endothelial']))] = 'Endothelial cells'



# Display the value counts of the new 'age_group' variable
print(adata.obs['age_cont'].value_counts())
def contains_zero_width_space(s):
    return '\u200b' in str(s)

# Iterate through observation variables and check for \u200b
variables_with_zero_width_space = []

for variable in adata.obs.columns:
    if adata.obs[variable].dtype.kind == 'O':  # Check if the variable is of object (string) type
        # Apply function only to non-null string values
        if any(adata.obs[variable].dropna().apply(lambda x: isinstance(x, str) and contains_zero_width_space(x))):
            variables_with_zero_width_space.append(variable)


# Display variables with \u200b
if variables_with_zero_width_space:
    print("Observation variables with \\u200b:")
    for variable in variables_with_zero_width_space:
        print(f"- {variable}")
else:
    print("No observation variables contain \\u200b.")



'study','Disease',

covariates = ['platform','viral_status_num',
	'sex_cont','age','Level_3_5','strain','total_counts',
	'pct_counts_mt']

int_types = ["integrated"]  # list needs to include "integrated" and/or "unintegrated"

if "unintegrated" in int_types:
    n_pcs = 30

min_n_cells_total = 50  # in total
min_n_cells = 10  # per sample
min_n_samples = 2

verbose = True


# Initiate a dictionary, in which we will store which samples were included per
# single regression
samples_included = dict()
cts_to_skip = list()  # we will fill this with cts that have too few samples
# these samples should be skipped in the rest of the analysis.
# Loop through specified integration types:
for int_type in int_types:
    samples_included[int_type] = dict()
    # loop through all annotations (= cell types), and also include the entire
    # atlas as a "cell type":
    for subset in sorted(adata.obs.Level_3_5.unique()) + ["whole_atlas"]:
        # remove space from cell type name, for file storing etc.
        subset_no_space = subset.replace(" ", "_")
        # initiate a dataframe, in which we will store for every covariate
        # which samples we include in the regression (we will exclude samples
        # that have no value available for the covariate under consideration
        # (e.g. BMI not recorded), as well as samples that have fewer than 10
        # cells of the cell type under consideration)
        samples_included[int_type][subset] = pd.DataFrame(
            index=adata.obs["sample"].unique(), columns=covariates
        )
        # if file doesn't exist yet (built in this check for if loop breaks halfway,
        # feel free to take it out):
        if not os.path.isfile(
            os.path.join(
                dir_results,
                f"variance_explained_fractions/variance_explained_fractions_{subset_no_space}_{int_type}.csv",
            )
        ):
            print(f"Working on {int_type}, {subset}...")
            # select the correct cells:
            if subset == "whole_atlas":
                subadata = adata.copy()
                verbose = True
            elif subset not in adata.obs.Level_3_5.unique():
                raise ValueError(
                    "subset should be set either to 'Whole atlas' or to a category in your manual_ann grouped obs variable!"
                )
            else:
                subadata = adata[adata.obs.Level_3_5 == subset, :].copy()
                verbose = False
            if subadata.n_obs < min_n_cells_total:
                print(f"{subset} has fewer than {min_n_cells_total} cells! Skipping.")
                continue
            # select the right embedding:
            if int_type == "unintegrated":
                emb_name = "X_pca"
                sc.tl.pca(subadata, n_comps=n_pcs, use_highly_variable=True)
            elif int_type == "integrated":
                emb_name = "X_MLCA"
            else:
                raise ValueError(
                    "emb_name should be set either to 'integrated' or 'unintegrated'"
                )
            # store the number of components in our embedding of choice
            n_comps = subadata.obsm[emb_name].shape[1]
            # initiate a dataframe in which we'll store the variance explained
            # by each covariate, plus the total variance ("overall") observed
            var_explained = pd.DataFrame(
                index=range(n_comps), columns=covariates + ["overall"]
            )
            # initiate a dataframe in which we will store the data
            # for our linear regression (i.e. the PC/latent components, + covariates).
            # Rows are cells, but we will collapse this to samples below
            comp_sample_df = pd.DataFrame(index=subadata.obs.index)
            comp_sample_df["sample"] = subadata.obs["sample"]
            # prepare aggregation dictionary for collapsing into sample-wise
            # observations
            agg_dict = {"sample": "count"}  # this will be the number of cells
            for comp in range(n_comps):
                # store component scores per cell
                comp_sample_df[f"comp{comp}"] = subadata.obsm[emb_name][:, comp]
                # we will aggregate these later by taking the mean per sample
                agg_dict[f"comp{comp}"] = "mean"
            for cov in covariates:
                if cov in ["log10_total_counts", "mito_frac"]:
                    # store values
                    comp_sample_df[cov] = subadata.obs[cov]
                    # we will aggregate by taking the mean
                    agg_dict[cov] = "mean"
                else:
                    # for all other covariates: these are sample-level
                    # covariates, so we will take the "first" observation
                    # in the sample (which should be the only)
                    comp_sample_df[cov] = subadata.obs[cov]
                    agg_dict[cov] = "first"
            # now collapse into sample-level observations
            sample_df = (
                comp_sample_df.groupby("sample")
                .agg(agg_dict)
                .rename(columns={"sample": "n_cells"})
            )
            print()
            # filter out samples with fewer than min_n_cells cells of the cell type
            for cov in covariates:
                if pd.api.types.is_categorical_dtype(sample_df[cov]):
                    sample_df[cov] = sample_df[cov].astype("category")
            sample_df = sample_df.loc[
                sample_df.n_cells >= min_n_cells,
            ].copy()
            # check number of samples left.
            # If fewer than min_n_samples remain, we will skip the cell type
            if sample_df.shape[0] < min_n_samples:
                print(
                    f"Only {sample_df.shape[0]} samples available for {subset}. Skipping."
                )
                cts_to_skip.append(subset)
                continue
            # Otherwise, move on to the linear regression:
            # do a linear regression on each component, with the component scores
            # as response variable...
            for comp in range(n_comps):
                # store the component values (for all samples i.e. unfiltered)
                y_true_unfiltered = sample_df.loc[:, f"comp{comp}"].values
                # and store variance of y_true as "overall" variance
                var_explained.loc[f"comp{comp}", "overall"] = np.var(y_true_unfiltered)
                # and the covariate as fixed variable
                for cov in covariates:
                    # store covariate observations under x
                    x = sample_df[cov].values.copy()
                    # store samples to which they match
                    x_samples = sample_df.index
                    # check which of these samples have no observation (e.g.
                    # because BMI was unknown, or age, etc.)
                    # (the function used below checks for different kinds of
                    # nas, e.g. np.nan, "nan", None, "None" etc.)
                    x_nans = np.vectorize(pd.isna)(x)
                    # now keep only xs that have real observations
                    x = x[~x_nans]
                    # if only one or no observations are left, skip this covariate
                    if len(x) < 2:
                        continue
                    # filter samples according to x filtering
                    x_samples = x_samples[~x_nans]
                    # and store which samples were included in our samples_included
                    # dictionary, for later reference (this is our "n")
                    samples_included[int_type][subset][cov] = samples_included[
                        int_type
                    ][subset].index.isin(x_samples.tolist())
                    # filter y_true according to x's filtering
                    y_true = y_true_unfiltered[~x_nans].reshape(-1, 1)
                    # prepare x for linear regression:
                    # if it is a float (e.g. BMI, age), all we need to do is reshape:
                    if x.dtype in ["float32", "float", "float64"]:
                        x = x.reshape(-1, 1)
                        # print that we are treating as numerical (only for first comp,
                        # so that we don't print the same thing many times)
                        if comp == 0 and verbose:
                            print(f"treating {cov} as continuous variable")
                    # otherwise we are dealing with a categorical...
                    else:
                        # if it has only one category, there is 0 variance and
                        # we cannot perform linear regression. In that case,
                        # move on to the next covariate.
                        if len(set(x)) == 1:
                            var_explained.loc[comp, cov] = np.nan
                            continue
                        # Otherwise, convert x to dummied variable:
                        # print that we are converting to dummy
                        # (only do it for the first comp, otherwise we print the same thing
                        # many times)
                        if comp == 0 and verbose:
                            print(f"converting {cov} to dummy variable")
                        # drop_first means we ensure that are x is full rank,
                        # and we only encode all-1 categories
                        x = pd.get_dummies(x, drop_first=True)
                    # now perform linear regression
                    lrf = LinearRegression(fit_intercept=True).fit(
                        x,
                        y_true,
                    )
                    # predict y based on the fit linear model
                    y_pred = lrf.predict(x)
                    # and store the variance of the predicted y, this is the
                    # "variance explained" by the covariate, for this component
                    var_explained.loc[comp, cov] = np.var(y_pred)
            # for each covariate, sum up how much variance it explains across
            # the components (i.e. PCs or scANVI latent components)
            # Sort covariates from explaining most to explaining least
            total_variance_explained = np.sum(var_explained, axis=0).sort_values(
                ascending=False
            )
            # divide this by the total variance that was observed in the
            # components, to get fraction of variance explained
            total_variance_explained_fractions = (
                total_variance_explained / total_variance_explained["overall"]
            )
            # write to files:
            # 1) variance explained fractions, for this integration type and cell type
            total_variance_explained_fractions.to_csv(
                os.path.join(
                    dir_results,
                    f"variance_explained_fractions/variance_explained_fractions_{subset_no_space}_{int_type}.csv",
                )
            )
            # 2) samples included, for this cell type
            samples_included[int_type][subset].to_csv(
                os.path.join(
                    dir_results,
                    f"samples_included/samples_included_{subset_no_space}.csv",
                )
            )


int_type = "integrated" # we used integrated in the paper, not unintegrated

subsets = ["whole_atlas"] + sorted(adata.obs.Level_3_5.unique())
# import variance explained fractions for the Whole atlas, to get indices
# and columns for dataframes
vef_df = pd.read_csv(
    os.path.join(
        dir_results,
        f"variance_explained_fractions/variance_explained_fractions_whole_atlas_{int_type}.csv",
    ),
    index_col=0,
).rename(columns={"0": "uncorrected"})

# inititate empty dataframes for variance explained fractions
vars_expl = pd.DataFrame(
    index=[subs for subs in subsets if subs not in cts_to_skip], columns=vef_df.index
)
# initiate an empty dictionary to store samples included for each cell type - cov
# pair
samples_included = dict()
# loop through cell types + Whole atlas
for subset in subsets:
    if subset not in cts_to_skip:
        # replace spaces in cell type name with "_" for file writing etc.
        subset_no_space = subset.replace(" ", "_")
        # import variance explained fractions:
        vef_df = pd.read_csv(
            os.path.join(
                dir_results,
                f"variance_explained_fractions/variance_explained_fractions_{subset_no_space}_{int_type}.csv",
            ),
            index_col=0,
        ).rename(columns={"0": "uncorrected"})
        # store values in our overview dataframes
        
        vars_expl.loc[subset, vef_df.index] = vef_df.values.reshape(1, -1)
        # store which samples were included for each covariate based on matching file
        samples_included[subset] = pd.read_csv(
            os.path.join(
                dir_results, f"samples_included/samples_included_{subset_no_space}.csv"
            ),
            index_col=0,
        )


# rename "whole_atlas" to "Whole atlas" index, for prettier plotting
vars_expl.rename(index={"whole_atlas": "Whole atlas"}, inplace=True)


cat_covs = [
    cat
    for cat in covariates
    if adata.obs[cat].values.dtype
    not in [
        "float32",
        "float",
        "float64",
    ]
]
# and continuous/numerical covariates
cont_covs = [
    cat
    for cat in covariates
    if adata.obs[cat].values.dtype
    in [
        "float32",
        "float",
        "float64",
    ]
]

# aggregate values by sample
agg_dict = {cov: "first" for cov in cat_covs}  # use "first" for categorical
agg_dict.update({cov: "mean" for cov in cont_covs})  # and "mean" for continuous
# collect statistics:
sample_df = adata.obs.groupby("sample").agg(agg_dict)
# initiate dataframe where we store for every cell type, for every covariate,
# how many unique values are observed
n_cats_per_cov = pd.DataFrame(
    index=[subs for subs in subsets if subs not in cts_to_skip],
    columns=cat_covs + cont_covs + ["sample"],
)

# loop through cell types and categorical covariates
for subset in subsets:
    if subset not in cts_to_skip:
        for cov in cat_covs:
            # annotation will always be one (this might change if we set our
            # cell type variable differently!)
            if cov == "Level_3_5":
                n_cats_per_cov[cov] = 1
            else:
                # extract covariate values only for samples included for cell type
                # and covariate
                sample_df_subset = sample_df.loc[
                    samples_included[subset].index[samples_included[subset][cov]], :
                ].copy()
                # count number of unique values
                n_per_cov = sample_df_subset[cov].nunique()
                # store
                n_cats_per_cov.loc[subset, cov] = n_per_cov
        # store number of samples included as the maximum number of samples included,
        # across covariates, for a cell type
        n_cats_per_cov.loc[subset, "sample"] = (
            samples_included[subset].sum(axis=0).max()
        )

# now create boolean df indicating in which cases only one unique value
# was observed for a cell type covariate pair
nis1 = n_cats_per_cov == 1
nis1.index = list(nis1.index)
nis1.loc["Whole atlas", :] = [
    adata.obs[cat_cov].nunique() == 1 for cat_cov in nis1.columns
]
#  now loop through continuous covariates and calculate variance instead of number
# of unique variables. If variance is 0, set nis1 to True.
for subset in subsets:
    if subset not in cts_to_skip:
        for cov in cont_covs:
            sample_df_subset = sample_df.loc[
                samples_included[subset].index[
                    (samples_included[subset].sum(axis=1) > 0)
                ],
                :,
            ].copy()
            variance = np.var(sample_df_subset[cov])
            if variance == 0 or np.isnan(variance):
                nis1.loc[subset, cov] = True
            else:
                nis1.loc[subset, cov] = False



nis1["overall"] = False


n_samples_included_df = pd.DataFrame(index=samples_included.keys(), columns=covariates)
for ct, samp_incl in samples_included.items():
    n_samples_included_df.loc[ct, :] = samp_incl.loc[:, covariates].sum(axis=0)

cts_ordered = [
    ct
    for ct in n_cats_per_cov.sort_values(by="sample", ascending=False).index.tolist()
    if ct != "whole_atlas"
]

technical_covariates = [
    "study",
    "platform",
    "pct_counts_mt",
    "Level_5",
    "total_counts"
]

# specify bio covariates to show
bio_covariates = [
	"viral_status_num",
    "sex_cont",
    "age",
	"Disease",
    "strain"
]

vars_expl.sort_values(by="Whole atlas", axis=1, ascending=False, inplace=True)
technical_covariates = [cov for cov in vars_expl.columns if cov in technical_covariates]
bio_covariates = [cov for cov in vars_expl.columns if cov in bio_covariates]
# now re-order vars_expl:
vars_expl = vars_expl.loc[
    ["Whole atlas"] + cts_ordered,
    technical_covariates + bio_covariates,
]

vars_expl_to_plot = vars_expl.copy()
# add capitalization
col_remapper = {
    col: (" ".join(col.replace("_", " ").split(" "))).capitalize()
    for col in vars_expl.columns
}
# remove "short" suffices

col_remapper["Level_3_5"] = "Cell type"
# We re-name some of the categories for clarity in the paper:
renaming = {
    "Group": "Mice conditions",
    "subject_type": "Subject status",
    "pct_counts_mt": "Fraction of mitochondrial UMIs",
    "viral_status_num": "Viral status",
}
col_remapper.update(renaming)
# now rename:
#vars_expl_to_plot.rename(columns=col_remapper, inplace=True)
# and remove dataset from figure
#vars_expl_to_plot.drop(columns="s", inplace=True)

mask1 = (
    n_samples_included_df.loc[
        ["whole_atlas"] + vars_expl.index[1:].tolist(),
        vars_expl.columns,
    ]
    < 5
)

# store mask for cell type - covariate pairs for which only 1 unique value
# was observed
mask2 = nis1.loc[
    ["whole_atlas"] + vars_expl.index[1:].tolist(),
    vars_expl.columns,
]

# combine masks
mask_overall = mask1 | mask2
# rename Whole atlas
mask_overall.rename(index={"whole_atlas": "Whole atlas"}, inplace=True)

#n_non_masked_rows = mask_overall.shape[0] - (mask_overall == True).sum(axis=0)
#columns_to_drop = n_non_masked_rows.loc[n_non_masked_rows < 5].index.tolist()

columns_to_drop=['viral_status_num']
#index_to_drop = ['Viral induced Gran','Viral induced monocytes']
vars_expl_to_plot.drop(columns=columns_to_drop, inplace=True)
#vars_expl_to_plot.drop(index=index_to_drop, inplace=True)

#mask_overall.drop(columns=columns_to_drop, inplace=True)

fz = 13  # fontsize
cmap = copy.copy(cm.get_cmap("Reds"))  # select color map
cmap.set_bad("grey")  # set masked color to grey instead of white
fig, ax = plt.subplots(figsize=(11, 11))  # figsize
vmax = 0.4
title = "Fraction of total variance"
# for file storing:
title_no_spaces = title.replace(" ", "_")
sns.heatmap(
    vars_expl_to_plot.astype(float),
    ax=ax,
    vmax=vmax,
    cbar_kws={"extend": "max"},
    mask=mask_overall.loc[vars_expl_to_plot.index, vars_expl_to_plot.columns].values,
    cmap=cmap,
)
ax.collections[0].colorbar.set_label(f"{title} explained", fontsize=fz + 4)
ax.set_xticklabels(ax.get_xmajorticklabels(),rotation=45, fontsize=fz+3)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=fz+3)
plt.title(
    f"{title} explained per covariate in integrated embedding\n(30 scANVI latent dimensions)",
    fontsize=fz+3,
)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=fz + 4)
plt.xlabel("Covariate (technical)             Covariate (biological)", fontsize=fz+3)
plt.ylabel("Cell type", fontsize=fz+3)
plt.tight_layout()
plt.grid(False)
plt.savefig('Control_Covariate_technical.png')
plt.show()
FIGURES[f"4a_{title_no_spaces}_explained_per_covariate_{int_type}_emb"] = fig


#find out what the variance is about, is the variance about cell composition 
#or cell something?

for subset in subsets:
    if subset not in cts_to_skip:
        cols = [col for col in cat_covs if col not in ["subject_ID", "Level_3_5"]]
        if subset not in "whole_atlas":
            plt.clf()
            plt.figure(figsize=(8, 6))
            plt.scatter(n_cats_per_cov.loc[subset, cols], vars_expl.loc[subset, cols])
            plt.xlabel("number of categories in cov")
            plt.ylabel("variance explained by cov")
            plt.title(subset)
            plt.tight_layout()
            plt.savefig(f"Variance_explained_by_covariance_{subset}.png")